# Mesh generation for tutorial 03

In [ ]:
import pygmsh
from dolfinx import *
from dolfinx.cpp.mesh import CellType, GhostMode
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Geometrical parameters

In [ ]:
r = 3
lcar = 1./4.

### Create mesh

In [ ]:
geom = pygmsh.built_in.Geometry()
p0 = geom.add_point([0.0, 0.0, 0.0], lcar)
p1 = geom.add_point([0.0, +r, 0.0], lcar)
p2 = geom.add_point([0.0, -r, 0.0], lcar)
c0 = geom.add_circle_arc(p1, p0, p2)
c1 = geom.add_circle_arc(p2, p0, p1)
l0 = geom.add_line(p2, p1)
geom.add_physical([c0, c1], label=1)
geom.add_physical(l0, label=2)
line_loop_left = geom.add_line_loop([c0, l0])
line_loop_right = geom.add_line_loop([c1, -l0])
semicircle_left = geom.add_plane_surface(line_loop_left)
semicircle_right = geom.add_plane_surface(line_loop_right)
geom.add_physical(semicircle_left, label=11)
geom.add_physical(semicircle_right, label=12)
pygmsh_mesh = pygmsh.generate_mesh(geom)
mesh = Mesh(MPI.comm_world, CellType.triangle, pygmsh_mesh.points[:, :2], pygmsh_mesh.cells["triangle"], [], GhostMode.none)

In [ ]:
plot(mesh)

### Create subdomains

In [ ]:
subdomains_mvc = MeshValueCollection("size_t", mesh, mesh.topology.dim, pygmsh_mesh.cells["triangle"], pygmsh_mesh.cell_data["triangle"]["gmsh:physical"] - 10)
subdomains = MeshFunction("size_t", mesh, subdomains_mvc, 0)

### Create boundaries

In [ ]:
boundaries_mvc = MeshValueCollection("size_t", mesh, mesh.topology.dim - 1, pygmsh_mesh.cells["line"], pygmsh_mesh.cell_data["line"]["gmsh:physical"])
boundaries = MeshFunction("size_t", mesh, boundaries_mvc, 0)

### Save

In [ ]:
with XDMFFile(MPI.comm_world, "circle.xdmf") as output:
    output.write(mesh)
with XDMFFile(MPI.comm_world, "circle_subdomains.xdmf") as output:
    output.write(subdomains)
with XDMFFile(MPI.comm_world, "circle_boundaries.xdmf") as output:
    output.write(boundaries)